In [1]:
from bs4 import BeautifulSoup
import json
import os

In [53]:
def parseHTMLtoJSON(html_filepath, json_filepath):
    # Парсим файл с помощью BeautifulSoup
    with open(html_filepath) as html:
        soup = BeautifulSoup(html, 'html.parser')

    # Находим заголовок
    title = soup.find('title').text

    # Находим link
    soup_link = soup.find_all('link', {'rel': 'canonical'})
    link = soup_link[0].get('href')

    # Находим содержимое статьи
    div_text = soup.find_all('div', {'class': 'wiki-content'})
    text = ""
    # Инициализируем необходимые тэги
    tags = ['p','td', 'h1', 'h2', 'th', 'li']
    # Итерируемся по строкам с содержимым тэгов
    for div in div_text:
        rows_text = div.find_all(tags)
        for row in rows_text:
            # Проверка на пустоту
            if len(row.text) > 0:
                # Добавляем содержимое в общий текст
                text += " " + row.text
    #Убираем переносы строк
    text = text.replace('\n', ' ')

    # Находим дату изменения
    date = ""
    soup_date = soup.find('a', {'class': 'last-modified'})
    if soup_date:
        date = soup_date.text

    # Находим авторов
    rows_author = soup.find_all('a', {'class': 'url fn'})
    author = ""
    # Итерируемся по строкам с содержимым тэгов
    for row in rows_author:
        # Проверка на пустоту
        if len(row.text) > 0:
            # Добавляем содержимое в общий текст
            author += row.text + ", "


    # Инициализируем объект с текстом и ссылкой на него (путь до файла)
    data = {'title' : title, 'text' : text, 'link' : link, 'date' : date, 'author' : author}
    # Открываем json
    with open(json_filepath,'r+') as file:
        # Загружаем данные из него
        file_data = json.load(file)
        # Добавляем в структуру pages новый объект
        file_data["pages"].append(data)
        # Возвращаем курсор в начало
        file.seek(0)
        # Загружаем обновленное содержимое
        json.dump(file_data, file, indent=4, ensure_ascii=False)

In [54]:
# Инициализируем путь до json файла
json_filepath = '/Users/vadim/Downloads/ConfluencePages.json'
pages = []
# Инициализируем объект с пустым массивом pages
data = {'pages':pages}
# Открываем json и записываем пустой массив
with open(json_filepath, 'w') as f:
        json.dump(data, f, indent=4)

In [55]:
# Инициализируем путь до директории с html файлами
dirpath = '/Users/vadim/Downloads/PECOM_ТГУ_TASK3'
# Итерируемся по директории
for root, dirs, files in os.walk(dirpath):
    # Итерируемся по файлам
    for file in files:
        # Проверка, что файл в формате html
        if file.endswith('.html'):
            # Инициализируем путь до итерируемого html файла
            html_filepath = os.path.join(root, file)
            # Парсим его в общий json
            parseHTMLtoJSON(html_filepath, json_filepath)